# Zadanie rekrutascyjne - Data Scientist - 3Soft

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split
import random 
import json
import yaml
from utilities.utl_eval import eval_metrics, preperation_data

import mlflow
import mlflow.sklearn

import logging

## Zadanie 1
Napisz funkcję 'dt_pred', która w oparciu o drzewo decyzyjne na podstawie losowego podzbioru obserwacji oraz losowego zestawu atrybutów ze zbioru uczącego zwraca wytrenowanie na podanym zbiorze treningowym drzewo decyzyjne. Odsetek obserwacji oraz odsetek atrybutów wykorzystywanych do uczenia drzewa powinny zostać sparametryzowane. Zadbaj o powtarzalność otrzymywanych wyników. 

In [ ]:
def dt_pred(X_data: pd.DataFrame, 
            y_data: pd.DataFrame,
            params: dict,
            seed: int=None):
    """
    Funkcja 'dt_pred', która w oparciu o drzewo decyzyjne na podstawie losowego podzbioru obserwacji oraz losowego zestawu atrybutów ze zbioru uczącego zwraca 
    wytrenowane na podanym zbiorze treningowym drzewo decyzyjne.
    Parameters:
        X_data: pd.DataFrame <- dane wejściowe do modelowania
        y_data: pd.DataFrame <- dane oczekiwane do modelowania
        params: dict  <- słownik parametrów (plik 'params.yaml')
        seed: int=None <- ziarno
    Return:
        modelTree <- wyuczone drzewo decyzyjne
        feature_col <- lista cech wykorzystanych podczas trenowania drzewa decyzyjnego
    """
    # Security
    if params['Bagging_atribute']['pct_of_training_samples'] < 0 or params['Bagging_atribute']['pct_of_training_samples'] > 1:
        raise ValueError("Feature `pct_of_training_samples` must have a value between 0 and 1.")
    if params['Bagging_atribute']['pct_of_features'] < 0 or params['Bagging_atribute']['pct_of_features'] > 1:
        raise ValueError("Feature `num_of_features` must have a value between 0 and 1 (these are the percentages).")
    # Data preparation
    if seed:
        np.random.seed(seed)
    else:
        np.random.seed(params['seed'])
    feature_col = np.random.choice(X_data.columns.values, 
                        round(params['Bagging_atribute']['pct_of_features']*len(X_data.columns.values)), 
                        replace=False)
    X_data = X_data[feature_col]
    X_train, X_test, y_train, y_test = train_test_split(X_data,  
                                                        y_data, 
                                                        train_size=params['Bagging_atribute']['pct_of_training_samples'], 
                                                        random_state=params['seed'],
                                                        )
    # Train model
    modelTree = DecisionTreeRegressor(criterion=params['DecisionTree']['criterion'], 
                                      max_depth=params['DecisionTree']['max_depth'],
                                      min_samples_split=params['DecisionTree']['min_samples_split'],
                                      random_state=seed)
    modelTree.fit(X_train, y_train)
    return modelTree, feature_col

## Zadanie 2
Następnie skonstruuj funkcję 'dt_bagg', która wykorzystywać będzie funkcję 'dt_pred' (jako tzw. weak learner) w procedurze baggingu*. Pamiętaj o uwzględnieniu odpowiednich (hiper)parametrów tej funkcji. Jeżeli to możliwe postaraj się zrównoleglić obliczenia. 

In [ ]:
class dt_bagg:
    def __init__(self,
                params: dict):
        """
        Klasa 'dt_bagg' została zaprojektowana jako algotyrm zespołowy (ensemble algorithm), który dopasowuje wiele modeli drzew decyzyjnych 
        do różnych podzbiorów danych treningowych, a następnie łączy przewidywania wszystkich modeli. Klasa przyjmuej jeden parametr: 
            params <- słownik parametrów (plik 'params.yaml') 
        Klasa zawiera 2 funkcje: 
            fit(X_train: pd.DataFrame, y_train: pd.DataFrame) <- wytrenowanie modelu
            predict(X_test: pd.DataFrame) <- predykcja wartości ciągłych dla danych testowych
        """
        self.params = params
        # Security
        if(self.params['RandomForest']['n_trees'] < 1):
            raise ValueError(f"Feature `n_trees` must have a integer greater or equal 1.")

    def fit(self, X_train: pd.DataFrame, y_train: pd.DataFrame):
        np.random.seed(self.params['seed'])
        list_of_seed = np.random.randint(0,10000, int(self.params['RandomForest']['n_trees']))
        self.trainedTree = [dt_pred(X_train, y_train, self.params, dt_random_state) for dt_random_state in list_of_seed]

    def predict(self, X_test: pd.DataFrame):
        y_pred = np.mean([self.trainedTree[idx][0].predict(X_test[self.trainedTree[idx][1]]) for idx in range(len(self.trainedTree))], axis=0)
        return y_pred



## Zadanie 3
Wykorzystaj napisaną przez siebie funkcję 'dt_bagg', aby na podstawie danych z załączonych plików (pliki 'signal.csv', 'time.csv' oraz 'descriptive.csv') zbudować odpowiednie modele prognozujące zmienną 'y' za pomocą zmiennych 'x_1'-'x_78'. Dokonaj podziału na zbiór treningowy oraz zbiór testowy losując 150 obserwacji.
Porównaj jakość otrzymanych prognoz w oparciu o: 
<ul>
  <li>Wykorzystanie funkcji 'dt_bagg' która zbuduje 100 niezależnych drzew, gdzie każde indywidualne drzewo uczone będzie na 80% obserwacji ze zbioru treningowego oraz 80% dostępnych atrybutów </li>
  <li>Wykorzystanie funkcji 'dt_bagg' która zbuduje 200 niezależnych drzew, gdzie każde indywidualne drzewo uczone będzie na 70% obserwacji ze zbioru treningowego oraz 50% dostępnych atrybutów </li>
</ul>
Oceń i porównaj na zbiorze testowym jakość prognoz otrzymanych za pomocą obu podejść.
Czy dobór (hiper)parametrów ma wpływ na otrzymywane wyniki? Czy jesteś w stanie zaproponować inny (lepszy) dobór tych parametrów?

In [ ]:
def train_model(dataset: dict, params: dict):
    """
    Funkcja do trenowania modeli na wyznaczonym zbiorze danych oraz o określionych hiperparametrach. Całość przygotowana pod MLflow Tracking.
    Parameters:
        dataset: dict <- słownik z danymi treningowymi i tesowymi
        params: dict <- słownik parametrów (plik 'params.yaml') 
    Return:
        output <- wyniki eksperymentu zapisane w folderze 'mlruns/0'. W celu zobaczenia oraz porówania przeprowadzonych eksperymentów w konsole 
            należy wpisać "$ mlflow ui" oraz przejść do okna przeglądarki po yrl: 'http://localhost:5000'
    """
    mlflow.set_tracking_uri("http://localhost:5000")
    
    with mlflow.start_run():
        if params['type_model'] == 'dt_bagg':
            reg_model = dt_bagg(params=params)
            reg_model.fit(dataset["train"]["X"], dataset["train"]["y"])
            y_pred_train = reg_model.predict(dataset["train"]["X"])
            y_pred_test = reg_model.predict(dataset["test"]["X"])
        elif params['type_model'] == 'BaggingRegression':
            reg_model = BaggingRegressor(
                base_estimator=DecisionTreeRegressor(
                    criterion=params['DecisionTree']['criterion'], 
                    max_depth=params['DecisionTree']['max_depth'],
                    min_samples_split=params['DecisionTree']['min_samples_split'],
                    random_state=params['seed']), 
                n_estimators=params['RandomForest']['n_trees'], 
                max_samples=params['Bagging_atribute']['pct_of_training_samples'], 
                max_features=params['Bagging_atribute']['pct_of_features'],
                bootstrap=False,
                oob_score=False,
                n_jobs=-1,
                random_state=params['seed']
            )
            reg_model.fit(dataset["train"]["X"], dataset["train"]["y"])
            y_pred_train = reg_model.predict(dataset["train"]["X"])
            y_pred_test = reg_model.predict(dataset["test"]["X"])

        rmse_train, mae_train, r2_train = eval_metrics(dataset['train']['y'], y_pred_train)
        rmse_test, mae_test, r2_test = eval_metrics(dataset['test']['y'], y_pred_test)

        mlflow.set_tag('mlflow.source.name', "-")
        mlflow.log_param("Type_model", params["type_model"])
        mlflow.log_param("criterion", params["DecisionTree"]['criterion'])
        mlflow.log_param("max_depth", params["DecisionTree"]['max_depth'])
        mlflow.log_param("min_samples_split", params["DecisionTree"]['min_samples_split'])
        mlflow.log_param("n_trees", params["RandomForest"]['n_trees'])
        mlflow.log_param("pct_of_features", params["Bagging_atribute"]['pct_of_features'])
        mlflow.log_param("pct_of_training_samples", params["Bagging_atribute"]['pct_of_training_samples'])
        mlflow.log_param("seed", params["seed"])

        mlflow.log_metric("train_RMSE", rmse_train)
        mlflow.log_metric("train_MAE",  mae_train)
        mlflow.log_metric("train_R2", r2_train)
        mlflow.log_metric("test_RMSE", rmse_test)
        mlflow.log_metric("test_MAE",  mae_test)
        mlflow.log_metric("test_R2", r2_test)



In [ ]:
with open("params.yaml", "r") as f:
    params = yaml.safe_load(f)

df_descriptive = pd.read_json('data/descriptive_v2.json')
main_dataset_dict = preperation_data(df_descriptive, params)


In [ ]:
idx = 0 
length = 2*3*5*3*2
for type_model in ['dt_bagg', 'BaggingRegression']:
    for criterion in ['squared_error', 'friedman_mse', 'absolute_error']:
        for max_depth in [4, 6, 8, 10, None]:
            for min_samples_split in [2, 3, 4]:
                for n_trees, pct_of_features, pct_of_training_samples in zip([100, 200], [0.8, 0.5], [0.8, 0.7]):
                    idx+=1
                    params['type_model'] = type_model
                    params['DecisionTree']['criterion'] = criterion
                    params['DecisionTree']['max_depth'] = max_depth
                    params['DecisionTree']['min_samples_split'] = min_samples_split
                    params['RandomForest']['n_trees'] = n_trees
                    params['Bagging_atribute']['pct_of_training_samples'] = pct_of_training_samples
                    params['Bagging_atribute']['pct_of_features'] = pct_of_features
                    print(f'Training model loading: {idx}/{length}')
                    train_model(dataset=main_dataset_dict,
                        params=params)

### Ocena otrzymanych eksperymentów

W przypadku tego zadania mamy do czynienia z problemem regresyjnym, a więc do porównywania jakości modeli wykorzytamy 3 miary jakości: $MAE$, $RMSE$, $R2$. Badane modele były testowane na następujących hiperparametrach:
- `criterion: ['squared_error', 'friedman_mse', 'absolute_error']` - funkcje do mierzenia jakości podziału;
- `max_depth: [4, 6, 8, 10, None]` - maksymalna głębokość drzewa
- `min_samples_split: [2, 3, 4]` - minimalna liczba próbek wymagana do podziału węzła

Dodatkowo w zadaniu oparłem się o model BaggingRegression, w celu porówaniania jakości otrzymanej regresji z modelem zaimplementowanym w bibliotece `sklearn`.

Wnioski:
1. Otrzymywane wyniki dochodzą do poziomu 0.27 $R^2$ na zbiorze testowym. Jednakże czasami zdarzają nam się obserwacje bliskie zera. Generalnie takie wyniki świadczą o tym, że nasza zmienna zależna (predykowana) jest ciężko wyjaśnialna przez nasze zmienne. 
2. W przygotowanym doświadczeniu dużo częściej lepiej poradziły sobie modele złożone ze 100 drzew decyzyjnych o wysokiej zawartości danych (80%/80%) niż te skłądające z większej liczby drzew decyzyjnych, ale słabszej reprezentacji danych (70%/50%). Przyczyną takiego stanu rzeczy może być fakt, że dużo cech było ze sobą silnie skorelowanych. Jeśli w ramach wyboru 50% wszystkich cech trafimy na zbiór składający się głównie z danych skorelowanych to jakość niesionej informacji jest bardzo niska. A to skutkuje otrzymaniem słabego modelu. Większa ilość drzew byłaby skudeczna, jeśli dane byłoby w duże części niezależne od siebie.
3. Dużą wadą otrzymanych modeli jest fakt, że dochodzi do przeuczenia. Dopasowanie $R^2$ na poziomie 0.9 na zbiorze treningowym oraz 0.2 na zbiorze testowym świadczy o tym, że albo mamy za mało danych albo wykorzystywany model jest za słaby dla tego zadania.
4. Dobór odpowiednich hiperparametrów modelu jest często cechą indywidualną dla danego zbioru danych, co zmusza nas często do poszukiwania odpowiednich hiperparametrów w ich pełmym zakresie. 
5. Model BaggingRegression poradził sobie w tym zadaniu trochę lepiej (0.309 $R^2$) niż zaimplementowana przeze mnie funkcja dt_bagg z głównego powodu, hiperparametr `max_feature` odpowiada maksymalnemu procentowi udział cech w procesie uczenia, czyli jeśli zajdzie taka potrzeba to model może do uczenia modelu może wykorzystać znacznie mniejsza ilość cech, gdzie w przypadku dt_bagg mamy sztywno określoną liczbę cech. Taka sama sytuacja występuje przy hiperparametrze `max_samples`.
6. Zaproponowanie dodatkowych hiperparametrów dla już istniejących funkcji nie zmieni znacznie otrzymywanych wyników. Chcąc poprawić jakość naszych predykcji należałoby zastanowić się:
- czy nie lepiej jest wybrać inną architekturę modelu (np. na zespoły pionowe tzw. modele boostingowe)?
- czy nie należałoby zwiększyć ilości danych (zarówno o nowe rekordy jak również nowe cechy)? np. rozbudować bazę w oparciu o szeregi czasowy
- w jaki sposób lepiej by było przeprowadzić naprawę naszych danych?